In [1]:
# 데이터 핸들링을 위한 패키지
import numpy as np
import pandas as pd

# 통계 학습을 위한 패키지
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 기계 학습을 위한 패키지
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 시각화를 위한 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# 그래프를 실제로 그리기 위한 설정
%matplotlib inline

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
news_df = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/06. 회귀분석 데이터/Composed_Data_ver1.4.csv', index_col = 0)
news_df.dropna(inplace = True)
news_df.reset_index(drop = True, inplace = True)
news_df.columns

Index(['URL', 'Publish Date', 'Title', 'Main Text', 'Ticker', 'Description',
       'News Type', 'Sentiment', 'Investor Reaction', 'GICS Sectors',
       'Firm Age', 'Twitter Followers', 'Total Asset', 'ROA',
       'Positive Consumer Perception', 'Negative Consumer Perception',
       'clean_text', 'truncated_text', 'ESG Bert Score', 'ESG Bert'],
      dtype='object')

In [3]:
news_df['GICS Sectors'].unique()

array(['Communication Services', 'Information Technology',
       'Consumer Discretionary', 'Health Care', 'Financials', 'Utilities',
       'Consumer Staples', 'Energy', 'Industrials', 'Real Estate',
       'Materials'], dtype=object)

In [4]:
sector = "Industrials"

In [5]:
news_df = news_df[['News Type','ESG Bert', 'Sentiment','GICS Sectors',
                   'Firm Age', 'Twitter Followers','Total Asset', 'ROA',
                   'Investor Reaction','Positive Consumer Perception', 'Negative Consumer Perception']]

In [6]:
news_df = news_df[news_df['Sentiment'] != "Neutral"]
news_df.reset_index(drop = True, inplace = True)

In [7]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3451 entries, 0 to 3450
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   News Type                     3451 non-null   object 
 1   ESG Bert                      3451 non-null   object 
 2   Sentiment                     3451 non-null   object 
 3   GICS Sectors                  3451 non-null   object 
 4   Firm Age                      3451 non-null   int64  
 5   Twitter Followers             3451 non-null   int64  
 6   Total Asset                   3451 non-null   float64
 7   ROA                           3451 non-null   float64
 8   Investor Reaction             3451 non-null   float64
 9   Positive Consumer Perception  3451 non-null   float64
 10  Negative Consumer Perception  3451 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 296.7+ KB


## 1. 변수들 표준화하기

In [8]:
#ROA Standard Scaling하기
ROA_array = np.array(news_df['ROA']).reshape(-1,1)
scaler1=StandardScaler()
scaler1.fit(ROA_array)
news_df['ROA'] = scaler1.transform(ROA_array)

#Total Asset Standard Scaling하기
TA_array = np.array(news_df['Total Asset']).reshape(-1,1)
scaler2=StandardScaler()
scaler2.fit(TA_array)
news_df['Total Asset'] = scaler2.transform(TA_array)

#Twitter Followers Standard Scaling하기
Twitter_array = np.array(news_df['Twitter Followers']).reshape(-1,1)
scaler3=StandardScaler()
scaler3.fit(Twitter_array)
news_df['Twitter Followers'] = scaler3.transform(Twitter_array)

#Firm Age Standard Scaling하기
FA_array = np.array(news_df['Firm Age']).reshape(-1,1)
scaler4=StandardScaler()
scaler4.fit(FA_array)
news_df['Firm Age'] = scaler4.transform(FA_array)

In [9]:
news_df

,News Type,ESG Bert,Sentiment,GICS Sectors,Firm Age,Twitter Followers,Total Asset,ROA,Investor Reaction,Positive Consumer Perception,Negative Consumer Perception
0,ESG,Data_Security,Negative,Communication Services,-0.674889,1.107763,-0.538118,-0.033619,0.064605,-0.019300,0.011234
1,Non-ESG,Employee_Engagement_Inclusion_And_Diversity,Positive,Information Technology,-0.502367,-0.747144,-0.551287,0.973663,-0.001291,-0.014514,0.004481
2,Financial Performance,Product_Design_And_Lifecycle_Management,Negative,Consumer Discretionary,-0.804280,0.934397,-0.537071,-0.574391,-0.010247,0.001597,-0.014112
3,Financial Performance,Systemic_Risk_Management,Negative,Information Technology,-0.222020,0.005162,0.419928,0.867669,-0.027306,-0.052604,0.017905
4,ESG,Business_Ethics,Negative,Communication Services,-0.674889,1.107763,-0.538118,-0.033619,0.042177,-0.004568,-0.009128
...,...,...,...,...,...,...,...,...,...,...,...
3446,ESG,Data_Security,Negative,Industrials,-0.868976,-0.699788,-0.543992,-2.425575,-0.009822,0.003075,-0.017754
3447,ESG,Management_Of_Legal_And_Regulatory_Framework,Positive,Communication Services,-0.631758,1.743807,0.504074,0.684711,-0.002094,-0.020018,0.028515
3448,ESG,Employee_Health_And_Safety,Negative,Industrials,-0.868976,-0.699788,-0.543992,-2.425575,0.018102,-0.011640,0.002777
3449,ESG,Labor_Practices,Positive,Communication Services,-0.847410,-0.778932,-0.558859,-0.013365,-0.014488,-0.056617,0.123931


## 2. 필요한 칼럼들 세팅하기

In [10]:
#news_df1: news sent 긍정, Consumer Perception 긍정
#news_df1: news sent 긍정, Consumer Perception 부정
#news_df1: news sent 부정, Consumer Perception 긍정
#news_df1: news sent 부정, Consumer Perception 부정
# news_df1 = pd.get_dummies(news_df, columns = ['Sentiment', 'GICS Sectors'])
# news_df1.drop(['Sentiment_Negative', 'Negative Consumer Perception',
#               'GICS Sectors_' + sector], axis = 1, inplace = True)

# news_df2 = pd.get_dummies(news_df, columns = ['Sentiment', 'GICS Sectors'])
# news_df2.drop(['Sentiment_Negative', 'Positive Consumer Perception',
#               'GICS Sectors_' + sector], axis = 1, inplace = True)

# news_df3 = pd.get_dummies(news_df, columns = ['Sentiment', 'GICS Sectors'])
# news_df3.drop(['Sentiment_Positive', 'Negative Consumer Perception',
#               'GICS Sectors_' + sector], axis = 1, inplace = True)

news_df4 = pd.get_dummies(news_df, columns = ['Sentiment', 'GICS Sectors'])
news_df4.drop(['Sentiment_Positive', 'Positive Consumer Perception',
              'GICS Sectors_' + sector], axis = 1, inplace = True)

In [11]:
news_df4['']

,News Type,ESG Bert,Firm Age,Twitter Followers,Total Asset,ROA,Investor Reaction,Negative Consumer Perception,Sentiment_Negative,GICS Sectors_Communication Services,GICS Sectors_Consumer Discretionary,GICS Sectors_Consumer Staples,GICS Sectors_Energy,GICS Sectors_Financials,GICS Sectors_Health Care,GICS Sectors_Information Technology,GICS Sectors_Materials,GICS Sectors_Real Estate,GICS Sectors_Utilities
0,ESG,Data_Security,-0.674889,1.107763,-0.538118,-0.033619,0.064605,0.011234,1,1,0,0,0,0,0,0,0,0,0
1,Non-ESG,Employee_Engagement_Inclusion_And_Diversity,-0.502367,-0.747144,-0.551287,0.973663,-0.001291,0.004481,0,0,0,0,0,0,0,1,0,0,0
2,Financial Performance,Product_Design_And_Lifecycle_Management,-0.804280,0.934397,-0.537071,-0.574391,-0.010247,-0.014112,1,0,1,0,0,0,0,0,0,0,0
3,Financial Performance,Systemic_Risk_Management,-0.222020,0.005162,0.419928,0.867669,-0.027306,0.017905,1,0,0,0,0,0,0,1,0,0,0
4,ESG,Business_Ethics,-0.674889,1.107763,-0.538118,-0.033619,0.042177,-0.009128,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446,ESG,Data_Security,-0.868976,-0.699788,-0.543992,-2.425575,-0.009822,-0.017754,1,0,0,0,0,0,0,0,0,0,0
3447,ESG,Management_Of_Legal_And_Regulatory_Framework,-0.631758,1.743807,0.504074,0.684711,-0.002094,0.028515,0,1,0,0,0,0,0,0,0,0,0
3448,ESG,Employee_Health_And_Safety,-0.868976,-0.699788,-0.543992,-2.425575,0.018102,0.002777,1,0,0,0,0,0,0,0,0,0,0
3449,ESG,Labor_Practices,-0.847410,-0.778932,-0.558859,-0.013365,-0.014488,0.123931,0,1,0,0,0,0,0,0,0,0,0


In [12]:
#종속변수 - 통제변수
news_df4_cont = news_df4.drop('Sentiment_Negative', axis = 1)
#종속변수 - 통제변수 + 독립변수
news_df4_ind = news_df4
#종속변수 - 통제변수 + 독립변수 + 조절변수
dummies = ['GICS Sectors_Communication Services',
       'GICS Sectors_Consumer Discretionary', 'GICS Sectors_Consumer Staples',
       'GICS Sectors_Energy', 'GICS Sectors_Financials',
       'GICS Sectors_Health Care', 'GICS Sectors_Information Technology',
       'GICS Sectors_Materials', 'GICS Sectors_Real Estate',
       'GICS Sectors_Utilities']
news_df4_mod = news_df4.copy()
for dummy in dummies:
    news_df4_mod[dummy + '_Moderate']  = news_df4_mod[dummy] * news_df4_mod['Total Asset']

In [16]:
news = news_df4_ind
#1. News
df_reg1 = news
#2. ESG News
df_reg2 = news[news['News Type'] == 'ESG']
#3. Non-ESG News
df_reg3 = news[(news['News Type'] == 'Non-ESG') | (news['News Type'] == 'Financial Performance')]
#4. Financial News
df_reg4 = news[news['News Type'] == 'Financial Performance']
#5. E News
df_reg5 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'GHG_Emissions') |
                                                     (news['ESG Bert'] == 'Air_Quality') |
                                                     (news['ESG Bert'] == 'Energy_Management') |
                                                     (news['ESG Bert'] == 'Ecological_Impacts') |
                                                     (news['ESG Bert'] == 'Water_And_Wastewater_Management') |
                                                     (news['ESG Bert'] == 'Waste_And_Hazardous_Materials_Management'))]
#6. S News
df_reg6 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'Customer_Privacy') |
                                                     (news['ESG Bert'] == 'Data_Security') |
                                                     (news['ESG Bert'] == 'Access_And_Affordability') |
                                                     (news['ESG Bert'] == 'Product_Quality_And_Safety') |
                                                     (news['ESG Bert'] == 'Customer_Welfare') |
                                                     (news['ESG Bert'] == 'Selling_Practices_And_Product_Labeling') |
                                                     (news['ESG Bert'] == 'Human_Rights_And_Community_Relations') |
                                                     (news['ESG Bert'] == 'Labor_Practices') |
                                                     (news['ESG Bert'] == 'Employee_Health_And_Safety') |
                                                     (news['ESG Bert'] == 'Employee_Engagement_Inclusion_And_Diversity')
                                                       )]

#7. G News
df_reg7 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'Business_Ethics') |
                                                     (news['ESG Bert'] == 'Competitive_Behavior') |
                                                     (news['ESG Bert'] == 'Systemic_Risk_Management') |
                                                     (news['ESG Bert'] == 'Critical_Incident_Risk_Management') |
                                                     (news['ESG Bert'] == 'Management_Of_Legal_And_Regulatory_Framework') |
                                                     (news['ESG Bert'] == 'Supply_Chain_Management') |
                                                     (news['ESG Bert'] == 'Business_Model_Resilience') |
                                                     (news['ESG Bert'] == 'Physical_Impacts_Of_Climate_Change') |
                                                     (news['ESG Bert'] == 'Director_Removal') |
                                                     (news['ESG Bert'] == 'Product_Design_And_Lifecycle_Management'))]
#8. Environment News
df_reg8 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'GHG_Emissions') |
                                                     (news['ESG Bert'] == 'Air_Quality') |
                                                     (news['ESG Bert'] == 'Energy_Management') |
                                                     (news['ESG Bert'] == 'Ecological_Impacts') |
                                                     (news['ESG Bert'] == 'Water_And_Wastewater_Management') |
                                                     (news['ESG Bert'] == 'Waste_And_Hazardous_Materials_Management'))]

#9. Social Capital News
df_reg9 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'Customer_Privacy') |
                                                     (news['ESG Bert'] == 'Data_Security') |
                                                     (news['ESG Bert'] == 'Access_And_Affordability') |
                                                     (news['ESG Bert'] == 'Product_Quality_And_Safety') |
                                                     (news['ESG Bert'] == 'Customer_Welfare') |
                                                     (news['ESG Bert'] == 'Selling_Practices_And_Product_Labeling') |
                                                     (news['ESG Bert'] == 'Human_Rights_And_Community_Relations'))]

#10. Human Capital News
df_reg10 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'Labor_Practices') |
                                                (news['ESG Bert'] == 'Employee_Health_And_Safety') |
                                                (news['ESG Bert'] == 'Employee_Engagement_Inclusion_And_Diversity'))]

#11. Leadership & Governance News
df_reg11 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'Business_Ethics') |
                                                     (news['ESG Bert'] == 'Competitive_Behavior') |
                                                     (news['ESG Bert'] == 'Systemic_Risk_Management') |
                                                     (news['ESG Bert'] == 'Critical_Incident_Risk_Management') |
                                                     (news['ESG Bert'] == 'Management_Of_Legal_And_Regulatory_Framework'))]

#12. Business Model & Innovation News
df_reg12 = news[(news['News Type'] == 'ESG') & ((news['ESG Bert'] == 'Supply_Chain_Management') |
                                                (news['ESG Bert'] == 'Business_Model_Resilience') |
                                                (news['ESG Bert'] == 'Physical_Impacts_Of_Climate_Change') |
                                                (news['ESG Bert'] == 'Director_Removal') |
                                                (news['ESG Bert'] == 'Product_Design_And_Lifecycle_Management'))]
#13. Non-Financial News
df_reg13 = news[news['News Type'] != 'Financial Performance']

In [18]:
# df_regs = [df_reg1,df_reg2,df_reg3,df_reg4,df_reg5,df_reg6,df_reg7,df_reg8,df_reg9,df_reg10,df_reg11,df_reg12]
df_regs = [df_reg13]
r_squared = []
f_pvalues = []
len_p_values = []
p_values = []
num_news = []

for reg in df_regs:
    #종속변수 - Investor Reaction
    df_reg = reg
    df_kc_reg = sm.add_constant(df_reg, has_constant='add')
    feature_columns = list(df_kc_reg.columns.difference(['News Type','ESG Bert','Investor Reaction', 'Negative Consumer Perception']))

    X = df_kc_reg[feature_columns]
    y = df_kc_reg['Investor Reaction']

    multi_linear_model = sm.OLS(y, X)
    result_model_1 = multi_linear_model.fit()
    
    r_squared.append(result_model_1.rsquared_adj)
    f_pvalues.append(result_model_1.f_pvalue)
    len_p_values.append(len(result_model_1.pvalues[result_model_1.pvalues < 0.05]))
    p_values.append(result_model_1.pvalues[result_model_1.pvalues < 0.05].index)
    num_news.append(len(reg))

In [19]:
df_i = pd.DataFrame(columns = ['Adj R-squared', 'Prob(F-statistic)','Num of P-value < 0.05', 'P-value < 0.05', 'Num of news'])
df_i['Adj R-squared'] = r_squared
df_i['Prob(F-statistic)'] = f_pvalues
df_i['P-value < 0.05'] = p_values
df_i['Num of P-value < 0.05'] = len_p_values
df_i['Num of news'] = num_news

In [20]:
df_i

,Adj R-squared,Prob(F-statistic),Num of P-value < 0.05,P-value < 0.05,Num of news
0,0.007016,0.002221,2,"Index(['GICS Sectors_Consumer Discretionary', ...",2895


In [21]:
df_i.to_excel('/Users/parkjunhyeong/Desktop/Regression_Data_Control_inv_ver1.0.xlsx', sheet_name = 'Non_financial_Inv_i')

In [22]:
# df_regs = [df_reg1,df_reg2,df_reg3,df_reg4,df_reg5,df_reg6,df_reg7,df_reg8,df_reg9,df_reg10,df_reg11,df_reg12]
df_regs = [df_reg13]
r_squared = []
f_pvalues = []
len_p_values = []
p_values = []
num_news = []

for reg in df_regs:
    #종속변수 - Investor Reaction
    df_reg = reg
    df_kc_reg = sm.add_constant(df_reg, has_constant='add')
    feature_columns = list(df_kc_reg.columns.difference(['News Type','ESG Bert','Investor Reaction', 'Negative Consumer Perception']))

    X = df_kc_reg[feature_columns]
    y = df_kc_reg['Negative Consumer Perception']

    multi_linear_model = sm.OLS(y, X)
    result_model_1 = multi_linear_model.fit()
    
    r_squared.append(result_model_1.rsquared_adj)
    f_pvalues.append(result_model_1.f_pvalue)
    len_p_values.append(len(result_model_1.pvalues[result_model_1.pvalues < 0.05]))
    p_values.append(result_model_1.pvalues[result_model_1.pvalues < 0.05].index)
    num_news.append(len(reg))

In [23]:
df_i = pd.DataFrame(columns = ['Adj R-squared', 'Prob(F-statistic)','Num of P-value < 0.05', 'P-value < 0.05', 'Num of news'])
df_i['Adj R-squared'] = r_squared
df_i['Prob(F-statistic)'] = f_pvalues
df_i['P-value < 0.05'] = p_values
df_i['Num of P-value < 0.05'] = len_p_values
df_i['Num of news'] = num_news

In [24]:
df_i

,Adj R-squared,Prob(F-statistic),Num of P-value < 0.05,P-value < 0.05,Num of news
0,0.032942,7.776137e-17,9,"Index(['Firm Age', 'GICS Sectors_Communication...",2895


In [25]:
df_i.to_excel('/Users/parkjunhyeong/Desktop/Regression_Data_Control_consumer_ver1.0.xlsx', sheet_name = 'Non_Financial_Consum_i')